In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score


In [2]:
df = pd.read_csv('food.csv')
df.head()

,Food_items,Breakfast,Lunch,Dinner,VegNovVeg,Calories,Fats,Proteins,Iron,Calcium,Sodium,Potassium,Carbohydrates,Fibre,VitaminD,Sugars
0,Asparagus Cooked,0,1,1,0,39,2.2,2.4,0.91,23.0,14,224.0,4.1,2.0,0.0,1.3
1,Avocados,1,0,0,0,160,15.0,2.0,0.55,12.0,7,485.0,8.5,6.7,0.0,0.7
2,Bananas,1,0,0,0,89,0.3,1.1,0.26,5.0,1,358.0,23.0,2.6,0.0,12.0
3,Bagels made in wheat,0,1,1,0,250,1.5,10.0,2.76,20.0,439,165.0,49.0,4.1,0.0,6.1
4,Berries,1,0,0,0,57,0.3,0.7,0.00,6.0,1,77.0,14.5,2.4,0.0,10.0


In [3]:
print('The shape of our features is:', df.shape)

The shape of our features is: (115, 16)


In [4]:
df.describe()

,Breakfast,Lunch,Dinner,VegNovVeg,Calories,Fats,Proteins,Iron,Calcium,Sodium,Potassium,Carbohydrates,Fibre,VitaminD,Sugars
count,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000
mean,0.478261,0.634783,0.773913,0.304348,189.573913,7.981739,10.920087,3.117217,66.015652,239.000000,262.160870,19.831043,2.414783,8.997391,6.516957
std,0.501713,0.483598,0.420127,0.462144,146.590075,11.006328,11.571388,7.621235,120.611064,304.458622,201.861012,21.642480,4.156927,52.114625,14.438641
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,85.500000,0.400000,2.600000,0.220000,7.000000,13.000000,134.500000,3.100000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,1.000000,0.000000,143.000000,3.100000,6.500000,1.000000,17.000000,80.000000,230.000000,14.000000,1.300000,0.000000,0.800000
75%,1.000000,1.000000,1.000000,1.000000,295.500000,11.100000,17.650000,2.805000,65.500000,378.000000,360.000000,28.600000,2.900000,0.000000,5.000000
max,1.000000,1.000000,1.000000,1.000000,600.000000,50.000000,80.000000,57.000000,683.000000,1531.000000,1154.000000,87.000000,34.000000,435.000000,82.000000


In [5]:
def cluster_food(FoodItemIDData, FoodItem_Test):
    ###### K-MEANS FOR ALL FOOD
    
    MealDatacalorie=FoodItemIDData[0:,2:len(FoodItemIDData)] #nutrion data
    MealDatacalorie = MealDatacalorie[:, [0,1,2,7,8]]

    # print(MealDatacalorie)
    S = StandardScaler()
    foods_scaled_data = S.fit_transform(MealDatacalorie)
    # print(foods_scaled_data)

    k_means_meals = KMeans(init="k-means++", n_clusters=3, n_init=50, max_iter=500, random_state=42)
    k_means_meals.fit(foods_scaled_data)
    labels=k_means_meals.labels_

    FoodItem_Test['KMCluster'] = labels
    
    # Overview data in clusters
    length = len(FoodItemIDData) + 2
    FoodItem_Test['KMCluster'] = labels
    clust_profile=FoodItem_Test.iloc[:,[2,3,4,9,10]].astype(float).groupby(FoodItem_Test['KMCluster']).mean()
    clust_profile['KMFrequency']=FoodItem_Test.KMCluster.value_counts().sort_index()
    clust = pd.DataFrame(clust_profile)
    print(clust)

    return labels

In [6]:
def food_data():
    FoodsData = df
    FoodsData=FoodsData.T 
    val=list(np.arange(5,16))
    Valapnd=[0]+[4]+val 
    FoodItemIDData=FoodsData.iloc[Valapnd]
    FoodItemIDData=FoodItemIDData.T
    
    return FoodItemIDData

In [7]:
FoodItemIDData = food_data()

FoodNutrion = FoodItemIDData

FoodItemIDData=FoodItemIDData.to_numpy()

foodlbs = cluster_food(FoodItemIDData, FoodNutrion)

FoodNutrion = FoodNutrion.drop(['Food_items'], axis = 1)
FoodNutrion = FoodNutrion.astype('float32')
print(FoodNutrion)
y = np.array(FoodNutrion['KMCluster'])
X= FoodNutrion.drop(['KMCluster','VegNovVeg','Iron', 'Calcium', 'Sodium', 'Potassium','VitaminD','Sugars'], axis = 1)
# features= FoodNutrion.drop(['KMCluster','Food_items','VegNovVeg'], axis = 1)
feature_list = list(X.columns)
X = np.array(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state=42)

             Calories       Fats   Proteins  Carbohydrates     Fibre  \
KMCluster                                                              
0          174.354839   7.267419  23.206452       3.122258  0.441935   
1           82.980769   2.140577   3.150192      13.534231  1.994231   
2          377.531250  18.165625  11.643750      46.250000  5.009375   

           KMFrequency  
KMCluster               
0                   31  
1                   52  
2                   32  
     VegNovVeg  Calories  Fats  Proteins  Iron  Calcium  Sodium  Potassium  \
0          0.0      39.0   2.2       2.4  0.91     23.0    14.0      224.0   
1          0.0     160.0  15.0       2.0  0.55     12.0     7.0      485.0   
2          0.0      89.0   0.3       1.1  0.26      5.0     1.0      358.0   
3          0.0     250.0   1.5      10.0  2.76     20.0   439.0      165.0   
4          0.0      57.0   0.3       0.7  0.00      6.0     1.0       77.0   
..         ...       ...   ...       ...   ...

In [8]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)


Training Features Shape: (80, 5)
Training Labels Shape: (80,)
Testing Features Shape: (35, 5)
Testing Labels Shape: (35,)


In [9]:
def train_hyper_tune(X,y):
    my_scaler = StandardScaler()
    my_imputer = SimpleImputer(strategy="median")
    
    ## Classifier 1: Decision Tree Classifier
    clf_DT = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    ## Classifier 2: Random Forest Classifier
    clf_RF = RandomForestClassifier(random_state=0)

    # create pipeline for each classifier
    ## clf_DT
    pipe1 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('dt_model',clf_DT)])
    ## clf_RF
    pipe2 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('rf_model',clf_RF)])
   
    
    # create hyperparameter
    ## clf_DT
    param_grid1 = {
        'dt_model__max_depth' : [4,8]
    }
    ## clf_RF
    param_grid2 = {
        'rf_model__n_estimators' : [50,100],
        'rf_model__max_features' : [0.8,"auto"],
        'rf_model__max_depth' : [4,8]
    }
    
    # set GridSearch
    ## clf_DT
    grid1 = GridSearchCV(pipe1, cv=5, param_grid=param_grid1)
    ## clf_RF
    grid2 = GridSearchCV(pipe2, cv=5, param_grid=param_grid2)
   
    grid1.fit(X,y)
    grid2.fit(X,y)
    
    return grid1,grid2,pipe1,pipe2

In [10]:
my_grid1,my_grid2,my_pipe1,my_pipe2 = train_hyper_tune(X_train, y_train)

In [11]:
my_grid1.best_score_

0.9375

In [13]:
my_grid2.best_score_

0.975

In [14]:
def train_on_entire(X,y,pipe,grid_res):
    pipe.set_params(**grid_res.best_params_).fit(X, y)
    
    return pipe

In [15]:
train_on_entire(X_train,y_train,my_pipe1,my_grid1).score(X_test,y_test)

0.9428571428571428

In [16]:
train_on_entire(X_train,y_train,my_pipe2,my_grid2).score(X_test,y_test)

0.9714285714285714

In [265]:
def Evaluation(X_train,y_train,X_test,y_test,pipe,grid_res):
    dt = pipe.set_params(**grid_res.best_params_).fit(X_train, y_train)
    dt_pred_test = dt.predict(X_test)
    print('Testing Set Evaluation F1-Score=>',f1_score(y_test,dt_pred_test,average='micro'))

In [266]:
Evaluation(X_train,y_train,X_test,y_test,my_pipe1,my_grid1)

Testing Set Evaluation F1-Score=> 0.9428571428571428


In [267]:
Evaluation(X_train,y_train,X_test,y_test,my_pipe2,my_grid1)

Testing Set Evaluation F1-Score=> 0.9714285714285714


In [278]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 50, num = 11)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            'max_features': max_features,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'bootstrap': bootstrap
            }

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)    # Fit the random search model
rf_random.fit(X_train, y_train)

rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'n_estimators': 50,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': True}

In [276]:
def evaluate(model, test_features, test_labels):
    y_pred=model.predict(test_features)
    print("Model Accuracy:", model.score(test_features, test_labels))

    return model.score(test_features, test_labels)

In [277]:
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test,y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test,y_test)

print('Improvement random grid of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Model Accuracy: 0.9714285714285714
Model Accuracy: 0.9428571428571428
Improvement random grid of -2.94%.


In [4]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rf.get_params())


Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}
